# SQL Alchemy Core: Part 1

## Overall Architecture

- The overall architecture is as follows:

**Frontend → API (e.g., Flask or FastAPI) → Backend (e.g., a server running Python) → SQLAlchemy Core (Reads) and SQLAlchemy ORM (Writes) → Database**

- According to [the SQLAlchemy documentation](https://docs.sqlalchemy.org) (2025):
  - **SQLAlchemy Core** is the foundational architecture for SQLAlchemy as a “database toolkit.” The library provides tools for managing connectivity to a database, interacting with database queries and results, and programmatic construction of SQL statements.
  - **SQLAlchemy ORM** builds upon SQLAlchemy Core to provide optional Object Relational Mapping (ORM) capabilities. This includes an additional configuration layer allowing user-defined Python classes to be mapped to database tables and other constructs, as well as an object persistence mechanism known as the Session. It then extends the Core-level SQL Expression Language to allow SQL queries to be composed and invoked in terms of user-defined objects.

## Install SQLAlchemy

- We need to install SQLAlchemy to work with databases in Python.

In [2]:
!pip install sqlalchemy

## Import SQLAlchemy

- We then import the SQLAlchemy library into the Python environment.
- Link to SQLAlchemy documentation: [https://docs.sqlalchemy.org](https://docs.sqlalchemy.org)

In [3]:
import sqlalchemy

The library contains several packages. An alternative way of importing the library is to import specific packages as needed:

In [40]:
from sqlalchemy import exc

Another alternative is to import all packages from SQLAlchemy:

In [5]:
from sqlalchemy import *

## Check the Version of SQLAlchemy

- We can confirm the version of SQLAlchemy installed using the `__version__` attribute.

In [6]:
sqlalchemy.__version__

'2.0.44'

## Create an Engine to Connect to the Database

- `create_engine()` is a function in SQLAlchemy that creates a new SQLAlchemy Engine instance. An Engine is the starting point for any SQLAlchemy application that wants to interact with a database. It manages the connection pool and provides a source of database connections.
- `echo=True` instructs SQLAlchemy to log all the SQL statements it executes to the console. This is useful for debugging and understanding what SQLAlchemy is doing behind the scenes.

In [7]:
engine = create_engine('sqlite+pysqlite:///mydatabase.db', echo=True)

## Connect to the Database using DataGrip

- Connect to the SQLite database using DataGrip (or any other similar software) to verify that the database file `mydatabase.db` has been created.
- In DataGrip, create a new data source and select SQLite. Then, select the `mydatabase.db` file to connect to the database.
- `mydatabase.db` should have been created in the current working directory when you executed the `create_engine()` function.

![img.png](assets/images/1_DataGripTOSQLLite.png)

## Connect to the Database using the engine in Python

In [8]:
conn = engine.connect()

## Execute Raw SQL Statements using SQLAlchemy Core

- We can also execute raw SQL statements using the connection object.
- Each SQL statement should be wrapped in the `text()` function from SQLAlchemy.
- The `commit()` method then commits any changes to the database.

In [9]:
conn.execute(text('CREATE TABLE IF NOT EXISTS supplier (name VARCHAR(255), rating int)'))

conn.commit()

2025-12-08 05:55:36,713 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 05:55:36,714 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS supplier (name VARCHAR(255), rating int)
2025-12-08 05:55:36,716 INFO sqlalchemy.engine.Engine [generated in 0.00287s] ()
2025-12-08 05:55:36,717 INFO sqlalchemy.engine.Engine COMMIT


- Confirm that the `supplier` table has been created in the `mydatabase.db` database using DataGrip or any other similar software.

![img.png](assets/images/2_CreateTableConfirmation.png)

In [10]:
conn.execute(text('INSERT INTO supplier (name, rating) VALUES ("Ruiru II Farm", 3);'))

conn.commit()

2025-12-08 05:55:36,751 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 05:55:36,752 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES ("Ruiru II Farm", 3);
2025-12-08 05:55:36,753 INFO sqlalchemy.engine.Engine [generated in 0.00220s] ()
2025-12-08 05:55:36,755 INFO sqlalchemy.engine.Engine COMMIT


- Confirm that the new record has been inserted.

![img.png](assets/images/3_Insert_Confirmation.png)

- It is recommended to limit the use of the connection object to a specific context. This can be done using a `with` statement, for example:

In [11]:
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2025-12-08 05:55:36,796 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 05:55:36,797 INFO sqlalchemy.engine.Engine select 'hello world'
2025-12-08 05:55:36,798 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ()
[('hello world',)]
2025-12-08 05:55:36,800 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM supplier"))
    print(result.all())

2025-12-08 05:55:36,828 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 05:55:36,829 INFO sqlalchemy.engine.Engine SELECT * FROM supplier
2025-12-08 05:55:36,830 INFO sqlalchemy.engine.Engine [generated in 0.00266s] ()
[('Ruiru II Farm', 3), ('Narok 41 Ranch', 4.8), ('Ngong Green House', 2), ('Makueni Fruits', 4), ('Pwani Farm', 1.8), ('Ruiru II Farm', 3)]
2025-12-08 05:55:36,833 INFO sqlalchemy.engine.Engine ROLLBACK


### Transaction Processing

#### Create
- Textual SQL is not the usual way we work with SQLAlchemy. However, when using textual SQL, a Python literal value, even non-strings like integers or dates, **should never be 'stringified' directly into an SQL string**; a parameter should always be used. This avoids SQL injection attacks.

In [13]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO supplier (name, rating) VALUES (:name, :rating)"),
        [{"name": "Narok 41 Ranch", "rating": 4.8}, {"name": "Ngong Green House", "rating": 2}],
    )
    conn.commit()

2025-12-08 05:55:36,868 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 05:55:36,869 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES (?, ?)
2025-12-08 05:55:36,870 INFO sqlalchemy.engine.Engine [generated in 0.00194s] [('Narok 41 Ranch', 4.8), ('Ngong Green House', 2)]
2025-12-08 05:55:36,872 INFO sqlalchemy.engine.Engine COMMIT


- Alternatively, we can use the `begin()` method to handle transactions.

  - `engine.connect()` creates a database connection that you can use to execute SQL statements. You must manually manage transactions (i.e., call `commit()` or `rollback()` as needed).
  - `engine.begin()` creates a connection and starts a transaction automatically. The transaction is committed if no exceptions occur, or rolled back if an exception is raised. This is useful for ensuring atomic operations and cleaner transaction management.

- Using `engine.begin()` is generally preferred for transaction management, as it reduces the risk of leaving transactions open unintentionally. However, if you need more control over the connection and transaction lifecycle, you can choose to use `engine.connect()`. This gives you access to the rollback() method if needed.

- Analogy: Driving a car with an automatic transmission (using `begin()`) vs. a manual transmission (using `connect()`).

![img.png](assets/images/4_Automatic-vs-Manual-Transmission.png)

In [14]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO supplier (name, rating) VALUES (:name, :rating)"),
        [{"name": "Makueni Fruits", "rating": 4}, {"name": "Pwani Farm", "rating": 1.8}],
    )
    conn.commit()

2025-12-08 05:55:36,902 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 05:55:36,904 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES (?, ?)
2025-12-08 05:55:36,905 INFO sqlalchemy.engine.Engine [cached since 0.03684s ago] [('Makueni Fruits', 4), ('Pwani Farm', 1.8)]
2025-12-08 05:55:36,907 INFO sqlalchemy.engine.Engine COMMIT


#### Read

Option 1: Default print output

In [30]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name, rating FROM supplier"))
    for row in result.fetchall():
        print(row)
    conn.commit()

2025-12-08 06:37:51,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 06:37:51,356 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier
2025-12-08 06:37:51,361 INFO sqlalchemy.engine.Engine [cached since 2448s ago] ()
('Ruiru II Farm', 3)
('Narok 41 Ranch', 4.9)
('Ngong Green House', 2)
('Makueni Fruits', 4.5)
2025-12-08 06:37:51,397 INFO sqlalchemy.engine.Engine COMMIT


Option 2: Customized print output

In [31]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name, rating FROM supplier"))
    for row in result.fetchall():
        print(f"name: {row.name},  rating: {row.rating}")
    conn.commit()

2025-12-08 06:38:04,991 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 06:38:04,994 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier
2025-12-08 06:38:04,996 INFO sqlalchemy.engine.Engine [cached since 2462s ago] ()
name: Ruiru II Farm,  rating: 3
name: Narok 41 Ranch,  rating: 4.9
name: Ngong Green House,  rating: 2
name: Makueni Fruits,  rating: 4.5
2025-12-08 06:38:05,003 INFO sqlalchemy.engine.Engine COMMIT


#### Update

In [32]:
with engine.connect() as conn:
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Makueni Fruits", "rating": 5}
    )
    conn.commit()

2025-12-08 06:38:17,933 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 06:38:17,934 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 06:38:17,935 INFO sqlalchemy.engine.Engine [cached since 1068s ago] (5, 'Makueni Fruits')
2025-12-08 06:38:17,939 INFO sqlalchemy.engine.Engine COMMIT


With multiple statements in one database transaction.

In [33]:
with engine.connect() as conn:
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Makueni Fruits", "rating": 4.5}
    )
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Narok 41 Ranch", "rating": 4.9}
    )
    result = conn.execute(
        text("SELECT name, rating FROM supplier WHERE name = :name"),
        {"name": "Makueni Fruits"}
    )
    for row in result.fetchall():
        print(row)
    conn.commit()

2025-12-08 06:38:29,108 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 06:38:29,111 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 06:38:29,114 INFO sqlalchemy.engine.Engine [cached since 1080s ago] (4.5, 'Makueni Fruits')
2025-12-08 06:38:29,119 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 06:38:29,120 INFO sqlalchemy.engine.Engine [cached since 1080s ago] (4.9, 'Narok 41 Ranch')
2025-12-08 06:38:29,122 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier WHERE name = ?
2025-12-08 06:38:29,123 INFO sqlalchemy.engine.Engine [cached since 717.3s ago] ('Makueni Fruits',)
('Makueni Fruits', 4.5)
2025-12-08 06:38:29,127 INFO sqlalchemy.engine.Engine COMMIT


#### Delete

In [34]:
with engine.connect() as conn:
    conn.execute(
        text("DELETE FROM supplier WHERE name = :name"),
        {"name": "Pwani Farm"}
    )

    conn.commit()

2025-12-08 06:38:42,060 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 06:38:42,061 INFO sqlalchemy.engine.Engine DELETE FROM supplier WHERE name = ?
2025-12-08 06:38:42,062 INFO sqlalchemy.engine.Engine [cached since 1031s ago] ('Pwani Farm',)
2025-12-08 06:38:42,069 INFO sqlalchemy.engine.Engine COMMIT


#### Rollback to a Savepoint
- A savepoint allows you to set a point within a transaction that you can roll back to without affecting the entire transaction. This is useful when you want to undo certain operations while keeping others intact.

In [38]:
with engine.connect() as conn:
    trans = conn.begin()  # start transaction
    try:
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Makueni Fruits", "rating": 4.3}
        )
        # Create a savepoint
        savepoint_1 = conn.begin_nested()
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 1.2}
        )
        # Rollback to savepoint (undo last update only)
        savepoint_1.rollback()
        result = conn.execute(
            text("SELECT name, rating FROM supplier WHERE name = :name"),
            {"name": "Makueni Fruits"}
        )
        for row in result.fetchall():
            print(row)
        trans.commit()
    except sqlalchemy.exc.SQLAlchemyError:
        trans.rollback()
        print("Transaction rolled back")

2025-12-08 08:48:31,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 08:48:31,584 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 08:48:31,591 INFO sqlalchemy.engine.Engine [cached since 8882s ago] (4.3, 'Makueni Fruits')
2025-12-08 08:48:31,619 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_1
2025-12-08 08:48:31,620 INFO sqlalchemy.engine.Engine [no key 0.00370s] ()
2025-12-08 08:48:31,625 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 08:48:31,626 INFO sqlalchemy.engine.Engine [cached since 8882s ago] (1.2, 'Narok 41 Ranch')
2025-12-08 08:48:31,635 INFO sqlalchemy.engine.Engine ROLLBACK TO SAVEPOINT sa_savepoint_1
2025-12-08 08:48:31,637 INFO sqlalchemy.engine.Engine [no key 0.00154s] ()
2025-12-08 08:48:31,640 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier WHERE name = ?
2025-12-08 08:48:31,642 INFO sqlalchemy.engine.Engine [cached since 8520s ago] ('Makueni Fruits',)
(

In [39]:
with engine.connect() as conn:
    trans = conn.begin()
    try:
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 0.5}
        )

        sp1 = conn.begin_nested()  # Savepoint 1
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Ngong Green House", "rating": 0.9}
        )

        sp2 = conn.begin_nested()  # Savepoint 2
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 5.0}
        )
        sp2.rollback()  # Undo the last update only
        sp1.rollback()  # Undo up to Savepoint 1 (including changes after sp1)
        trans.commit()
    except sqlalchemy.exc.SQLAlchemyError:
        trans.rollback()

2025-12-08 08:48:48,714 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 08:48:48,716 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 08:48:48,717 INFO sqlalchemy.engine.Engine [cached since 8899s ago] (0.5, 'Narok 41 Ranch')
2025-12-08 08:48:48,721 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_1
2025-12-08 08:48:48,722 INFO sqlalchemy.engine.Engine [no key 0.00130s] ()
2025-12-08 08:48:48,723 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 08:48:48,724 INFO sqlalchemy.engine.Engine [cached since 8899s ago] (0.9, 'Ngong Green House')
2025-12-08 08:48:48,727 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_2
2025-12-08 08:48:48,727 INFO sqlalchemy.engine.Engine [no key 0.00096s] ()
2025-12-08 08:48:48,729 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 08:48:48,729 INFO sqlalchemy.engine.Engine [cached since 8899s ago] (5.0, 'Narok 41 Ranch')
2025-12-0

## The MetaData Class

- The previous chunks show the most basic way of working with SQL Alchemy Core. In practice, you would typically define classes that map to your database tables and use the ORM features to work with those classes instead of writing raw SQL queries. This allows for a more **Pythonic** way of interacting with the database.
- The MetaData class is used to define the database schema. It keeps track of all the information necessary to create tables, constraints, and other database objects.

## Close the Connection
- It is good practice to close the connection when you are done. This also commits anything that has not yet been committed up to this point.

In [8]:
conn.close()

## References
SQLAlchemy Project. (2025, December 5). _SQLAlchemy 2.0 Documentation._ SQLAlchemy. https://docs.sqlalchemy.org/en/20/
